# Analysis for the experiments on the SINC scenario

In [1]:
# Import all libraries
import pandas as pd

In [2]:
# Settings
fileName = "resultsSINC.csv"
orderingStrategyPMedici = "IN_ORDER_SIZE_DESC"

In [3]:
# Load the file into a pandas dataframe
df = pd.read_csv(fileName, names=['Generator','ModelName','PercentageOfOldTsKept','TSSize','TSTime','Strength','Ordering'], header=0, index_col=False)
# Set Ordering to empty string where it is NaN
df['Ordering'].fillna('', inplace=True)

C:\Users\Andrea_PC\AppData\Local\Temp\ipykernel_8828\1893749315.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(fileName, names=['Generator','ModelName','PercentageOfOldTsKept','TSSize','TSTime','Strength','Ordering'], header=0, index_col=False)


## Group all executions

In [6]:
# Group data in df by Generator, ModelName, PercentageOfOldTsKept, Strength, and Ordering. For the TSSize and TS time, take the mean
df = df.groupby(['Generator','ModelName','PercentageOfOldTsKept','Strength','Ordering']).agg({'TSSize':'mean', 'TSTime':'mean'}).reset_index()
# Remove from df all the lines where generator is pMEDICI or pMEDICI+ but Ordering is not equal to orderingStrategyPMedici
dfFiltered = df[~(df['Generator'].str.contains('pMEDICI')) | (df['Generator'].str.contains('pMEDICI') & (df['Ordering'] == orderingStrategyPMedici))]
print (dfFiltered)

    Generator          ModelName  PercentageOfOldTsKept  Strength  \
0        ACTS            BOOLC_0                      0         2   
1        ACTS            BOOLC_0                    100         3   
2        ACTS            BOOLC_1                      0         2   
3        ACTS            BOOLC_1                    100         3   
4        ACTS            BOOLC_2                      0         2   
..        ...                ...                    ...       ...   
632  pMEDICI+  UNIFORM_BOOLEAN_5                      0         3   
636  pMEDICI+  UNIFORM_BOOLEAN_6                      0         3   
640  pMEDICI+  UNIFORM_BOOLEAN_7                      0         3   
644  pMEDICI+  UNIFORM_BOOLEAN_8                      0         3   
648  pMEDICI+  UNIFORM_BOOLEAN_9                      0         3   

               Ordering  TSSize   TSTime  
0                          10.0   1114.4  
1                          22.0  16287.8  
2                          12.0     59.6  